# ACSVHomotopy
## Setup
Import and activate the locally stored ACSVHomotopy package. The package reexports `@polyvar` along with functionality from HomotopyContinuation for working with polynomials.

In [1]:
# If you don't already have HomotopyContinuation.jl installed this might take a while
using Pkg
Pkg.activate(".")
using ACSVHomotopy

  Activating project at `~/Desktop/Projects/ACSVMath-ACSVHomotopy`
┌ Info: Precompiling ACSVHomotopy [aec64979-1285-4ff6-b7ad-06ab3746491a]
└ @ Base loading.jl:1423


## Example Usage
Consider first the rational function
$$ F(x,y,z) = \frac{1}{1-(1+z)(x+y-xy)},$$
whose main diagonal is related to one of Apéry's irrationality proofs.

In [2]:
# Define the polynomial
@polyvar x y z
H = 1-(1+z)*(x+y-x*y)

xyz + xy - xz - yz - x - y + 1

In [3]:
# Smooth critical points can easily be approximated by solving the smooth critical point system.
# Note: This step is not necessary to use our methods, we just include it for discussion.
function get_crits(h)
    vars = variables(h)
    n = length(vars)
    @polyvar λ
    sys = System([h; vars .* differentiate(h, vars) - λ], variables=[vars; λ])
    return [sol[1:end-1] for sol in solutions(solve(sys, show_progress=false))]
end

# In this case, we get three critical points. But which (if any) is minimal?
crits = get_crits(H)

3-element Vector{Vector{ComplexF64}}:
 [0.38196601125010526 + 3.76158192263132e-37im, 0.38196601125010526 + 1.9983403963978888e-37im, 0.6180339887498947 + 1.88079096131566e-37im]
 [2.618033988749895 + 3.851859888774472e-34im, 2.618033988749895 - 3.851859888774472e-34im, -1.6180339887498947 - 2.407412430484045e-35im]
 [1.0 + 0.0im, 1.0 + 0.0im, -2.5632011150913368e-17 + 0.0im]

In [4]:
# Knowing that F is combinatorial we can get the minimal critical point as follows
min_cp = find_min_crits_comb(H)

1-element Vector{Vector{Float64}}:
 [0.3819660112501051, 0.3819660112501051, 0.6180339887498949]

In [5]:
# Knowing the minimal critical point, we can get the leading asymptotic term
# The exponential growth implies irrationality of zeta(2) (which can also be shown using other methods)
leading_asymptotics(1,H,min_cp)

"(0.09016994374947422)^(-n) n^(-1.0) (0.3544432672196469 + 0.0im)"

In [6]:
# It's not obvious from the definition that F is combinatorial. If we can't assume this, 
# then we can use the (much more expensive) find_min_crits command.
# WARNING: This cell takes 10 - 20 MINUTES to run
# We pass the show_progress=true flag to see the progess of the homotopy solver as it runs
find_min_crits(H; show_progress=true)

Tracking 36720 paths... 100%|███████████████████████████| Time: 0:09:13
  # paths tracked:                  36720
  # non-singular solutions (real):  16 (0)
  # singular endpoints (real):      60 (0)
  # total solutions (real):         76 (0)


1-element Vector{Vector{Float64}}:
 [0.3819660112501051, 0.3819660112501051, 0.6180339887498949]

In [7]:
# The general code is very expensive. If we want to use the numeric approximation approach
# of the paper, we can obtain the same point in a less rigorous manner much faster.
find_min_crits(H, approx_crit=true)

1-element Vector{Vector{Float64}}:
 [0.3819660112501051, 0.3819660112501051, 0.6180339887498949]

In [8]:
# We can also use the monodromy approach from the paper
find_min_crits(H, monodromy=true)

1-element Vector{Vector{Float64}}:
 [0.3819660112501051, 0.3819660112501051, 0.6180339887498949]

## Benchmarking
We now run the benchmark examples used in the paper. Note that unlike the paper **here we include compile time**. You can see how much of the runtime was spent on the compilation in the timing output.

In [9]:
# Define polynomial variables for examples
@polyvar w x y z;

In [10]:
# Binomial coefficients
H = 1 - x - y
@time find_min_crits_comb(H);
@time find_min_crits(H);
@time find_min_crits(H; approx_crit=true);
@time find_min_crits(H; monodromy=true);

  7.982619 seconds (12.49 M allocations: 625.723 MiB, 2.16% gc time, 99.81% compilation time)
 12.133083 seconds (19.17 M allocations: 943.212 MiB, 2.25% gc time, 99.62% compilation time)
 12.448644 seconds (18.73 M allocations: 939.023 MiB, 4.74% gc time, 99.71% compilation time)
  5.754329 seconds (18.40 M allocations: 845.835 MiB, 3.88% gc time, 59.63% compilation time)


In [11]:
# Melczer Salvy example of two positive real critical points
H = (1-x-y)*(20-x-40*y)-1 
@time find_min_crits_comb(H);
@time find_min_crits(H);
@time find_min_crits(H; approx_crit=true);
@time find_min_crits(H; monodromy=true);

  8.155952 seconds (12.01 M allocations: 600.434 MiB, 2.23% gc time, 99.61% compilation time)
 17.542391 seconds (22.37 M allocations: 1.053 GiB, 1.75% gc time, 76.69% compilation time)
 12.784650 seconds (19.43 M allocations: 955.091 MiB, 3.72% gc time, 95.25% compilation time)
 10.833561 seconds (22.14 M allocations: 1.043 GiB, 3.87% gc time, 84.78% compilation time)


In [12]:
# Related to asymptotics of sqrt(1-z)
H = 1 - x*y - x*y^2 - 2*x^2*y
@time find_min_crits_comb(H);
@time find_min_crits(H);
@time find_min_crits(H; approx_crit=true);
@time find_min_crits(H; monodromy=true);

  7.808429 seconds (11.79 M allocations: 587.296 MiB, 1.98% gc time, 99.65% compilation time)
 43.822623 seconds (26.23 M allocations: 1.210 GiB, 0.91% gc time, 33.37% compilation time)
 13.482531 seconds (20.65 M allocations: 1005.796 MiB, 3.65% gc time, 94.92% compilation time)
 20.561036 seconds (39.19 M allocations: 1.689 GiB, 2.72% gc time, 47.88% compilation time)


In [13]:
# Gillis-Reznick-Zeilberger function
H = 1 - (x+y+z) + 5*x*y*z
@time find_min_crits(H);
@time find_min_crits(H; approx_crit=true);
@time find_min_crits(H; monodromy=true);

Computing mixed cells... 1079 	 Time: 0:00:00
  mixed_volume:  13068


284.918124 seconds (41.79 M allocations: 1.772 GiB, 0.20% gc time, 6.74% compilation time)
 16.961707 seconds (21.55 M allocations: 1.019 GiB, 1.48% gc time, 76.28% compilation time)
 17.369396 seconds (26.15 M allocations: 1.208 GiB, 1.87% gc time, 57.32% compilation time)


In [14]:
# Apéry zeta(2)
H = 1-(1+z)*(x+y-x*y)
@time find_min_crits(H);
@time find_min_crits(H; approx_crit=true);
@time find_min_crits(H; monodromy=true);

Computing mixed cells... 2283 	 Time: 0:00:01
  mixed_volume:  36720


748.686942 seconds (21.28 M allocations: 576.698 MiB, 0.01% gc time, 0.02% compilation time)
  3.387627 seconds (684.34 k allocations: 30.031 MiB, 1.05% gc time)
  8.432320 seconds (32.46 M allocations: 1.385 GiB, 7.53% gc time, 35.01% compilation time)


In [15]:
# Random poly of degree 4
H = 1 - (72*x^3*z + 97*y*z^3 + 53*x*z^2 + 47*x*y + 39*z^2 + 71*x)
@time find_min_crits_comb(H);
@time find_min_crits(H; approx_crit=true);
@time find_min_crits(H; monodromy=true);

  8.416391 seconds (12.50 M allocations: 618.293 MiB, 2.33% gc time, 98.28% compilation time)
204.055742 seconds (37.17 M allocations: 1.618 GiB, 0.22% gc time, 6.86% compilation time)
480.229058 seconds (609.06 M allocations: 22.026 GiB, 1.06% gc time, 2.54% compilation time)


In [16]:
# 2D lattice path example
H = 1 - z*x^2*y - z*x*y^2 - z*x - z*y
@time find_min_crits_comb(H);
@time find_min_crits(H; approx_crit=true);
@time find_min_crits(H; monodromy=true);

  7.944972 seconds (12.18 M allocations: 602.189 MiB, 2.10% gc time, 99.46% compilation time)
 31.051635 seconds (26.95 M allocations: 1.253 GiB, 1.91% gc time, 47.85% compilation time)
 38.276381 seconds (181.05 M allocations: 6.868 GiB, 4.01% gc time, 6.81% compilation time)


In [17]:
# 3D lattice path example
H = 1 - w*x^2*y*z - w*x*y^2*z - w*x*y*z^2 - w*x*y - w*x*z - w*y*z
@time find_min_crits_comb(H);

  8.467614 seconds (13.20 M allocations: 644.548 MiB, 1.72% gc time, 99.14% compilation time)


In [18]:
# High degree example
@polyvar x y w z
H = 1-x-y^2-w^3-z^4
@time find_min_crits_comb(H);

  8.176509 seconds (12.13 M allocations: 605.657 MiB, 2.42% gc time, 98.15% compilation time)


In [19]:
# Apéry zeta(3)
H = 1 - w*(1 + x)*(1 + y)*(1 + z)*(x*y*z + y*z + y + z + 1) 
@time find_min_crits_comb(H);

 10.343989 seconds (15.68 M allocations: 751.245 MiB, 2.30% gc time, 89.70% compilation time)
